In [ ]:
from IPython.display import Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2, os
import numpy as np

def take_photo(filename='photo.jpg', quality=0.8):
    js = '''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = '📸 Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        div.appendChild(video);

        document.body.appendChild(div);

        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks().forEach(track => track.stop());
        div.remove();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        return dataUrl;
      }
      takePhoto({quality: %f});
    ''' % quality
    data = eval_js(js)
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [ ]:
os.makedirs("dataset/me", exist_ok=True)
os.makedirs("dataset/friend", exist_ok=True)


In [ ]:
#For me
for i in range(1, 11):   # change 11 → 51 if you want 50 photos
    filename = take_photo(f"dataset/me/me_{i}.jpg")
    print("Saved", filename)


Saved dataset/me/me_1.jpg
Saved dataset/me/me_2.jpg
Saved dataset/me/me_3.jpg
Saved dataset/me/me_4.jpg
Saved dataset/me/me_5.jpg
Saved dataset/me/me_6.jpg
Saved dataset/me/me_7.jpg
Saved dataset/me/me_8.jpg
Saved dataset/me/me_9.jpg
Saved dataset/me/me_10.jpg


In [ ]:
#Take your images through webscam
for i in range(1, 11):   # change 11 → 51 if you want 50 photos
    filename = take_photo(f"dataset/friend/friend_{i}.jpg")
    print("Saved", filename)


Saved dataset/friend/friend_1.jpg
Saved dataset/friend/friend_2.jpg
Saved dataset/friend/friend_3.jpg
Saved dataset/friend/friend_4.jpg
Saved dataset/friend/friend_5.jpg
Saved dataset/friend/friend_6.jpg
Saved dataset/friend/friend_7.jpg
Saved dataset/friend/friend_8.jpg
Saved dataset/friend/friend_9.jpg
Saved dataset/friend/friend_10.jpg


In [ ]:
#Take friend images through web
import glob
print("My images:", len(glob.glob("dataset/me/*.jpg")))
print("Friend images:", len(glob.glob("dataset/friend/*.jpg")))


My images: 10
Friend images: 10


In [ ]:
#Preprocess images
import cv2
import numpy as np
import glob

IMG_SIZE = 128
X, y = [], []

def load_images(folder, label):
    data, labels = [], []
    for file in glob.glob(folder + "/*.jpg"):
        img = cv2.imread(file)
        if img is None:  # skip bad images
            continue
        # Face detection (optional but better)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) > 0:
            x,yc,w,h = faces[0]
            img = img[yc:yc+h, x:x+w]  # crop face

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data.append(img)
        labels.append(label)
    return data, labels

me_data, me_labels = load_images("dataset/me", 0)       # Label 0 = Me
friend_data, friend_labels = load_images("dataset/friend", 1) # Label 1 = Friend

X = np.array(me_data + friend_data) / 255.0
y = np.array(me_labels + friend_labels)

print("Dataset shape:", X.shape, "Labels:", y.shape)


Dataset shape: (20, 128, 128, 3) Labels: (20,)


In [ ]:
#Train-Test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5625 - loss: 0.6947 - val_accuracy: 0.7500 - val_loss: 0.4956
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 858ms/step - accuracy: 0.5625 - loss: 0.6502 - val_accuracy: 0.5000 - val_loss: 1.6297
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - accuracy: 0.5000 - loss: 1.5448 - val_accuracy: 1.0000 - val_loss: 0.3360
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step - accuracy: 0.8750 - loss: 0.2949 - val_accuracy: 0.5000 - val_loss: 0.7095
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - accuracy: 0.6250 - loss: 0.5629 - val_accuracy: 0.7500 - val_loss: 0.4161
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step - accuracy: 0.8750 - loss: 0.2831 - val_accuracy: 1.0000 - val_loss: 0.1447
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step - accuracy: 1.0000 - loss: 0.0680 - val_accuracy: 1.0000 - val_loss: 0.1660
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step - accuracy: 1.0000 - loss: 0.0780 - val_accuracy: 1.0000 - val_loss: 0.

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 1.0000 - loss: 0.0431
Test Accuracy: 100.00%


In [35]:
filename = take_photo("test.jpg")

img = cv2.imread(filename)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = np.expand_dims(img/255.0, axis=0)

prediction = model.predict(img)[0][0]

# Add threshold logic
if prediction < 0.3:
    print("Prediction: It's YOU 😃")
elif prediction > 0.7:
    print("Prediction: It's YOUR FRIEND 👯")
else:
    print("Prediction: It's a DIFFERENT PERSON 🕵️")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: It's a DIFFERENT PERSON 🕵️
